In [4]:
import os
import requests
import tiktoken
import numpy as np
from tqdm import tqdm
from datasets import load_dataset # huggingface datasets

# download the tiny shakespeare dataset or load it from a local file
# os.getcwd() returns the current working directory
input_file_path = os.path.join(os.getcwd(), 'input.txt')
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w') as f:
        f.write(requests.get(data_url).text)
    
dataset = load_dataset("text", data_files="input.txt")

# input.txt by default only contains the 'train' split, so create a test split
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=2357, shuffle=True)
split_dataset['val'] = split_dataset.pop('test') # rename the test split to val

print(split_dataset.items(), '\n')


# split_dataset['train'][0 ~ 35999]['text'] = "often give us ..."
# split_dataset['val'][0 ~ 3999]['text'] = "thee, and ..."

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 40000
    })
}) 

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4000
    })
})


In [28]:
print(split_dataset['train'][10]['text'])
print(split_dataset['train'][:])


often give us soldiers the lie: but we pay them for
{'text': ['', '', "I'll answer thee in any fair degree,", "No, first let's reason with him.", "With what's unreal thou coactive art,", 'I would not lose the dog for twenty pound.', 'As I have set them down. If I do send, dispatch', 'Clown:', 'As if the golden fee for which I plead', '', 'often give us soldiers the lie: but we pay them for', "If that be sin, I'll make it my morn prayer", 'Nothing that I respect, my gracious lord.', 'Second Citizen:', 'What fit is this, good lady?', "Perchance she cannot meet him: that's not so.", 'Where shivering cold and sickness pines the clime;', 'Ay, as the glasses where they view themselves;', 'Let us entreat you stay till after dinner.', 'Make me to think so twenty years together!', 'Suitors to her and rivals in my love,', 'And till she stoop she must not be full-gorged,', 'Please but your honour hear me.', 'Three times to-day my foot-cloth horse did stumble,', 'To insinuate, flatter, bow, and be

In [16]:
num_proc = 2
enc = tiktoken.get_encoding("gpt2")

def process(example):
    ids = enc.encode_ordinary(example['text'])
    ids.append(enc.eot_token)
    out = {'ids': ids, 'len': len(ids)}
    return out

# tokenize the dataset
tokenized = split_dataset.map(
    process,
    remove_columns=['text'],
    desc="tokenizing the splits",
    num_proc=num_proc,
)
print(split_dataset.items(), '\n')
print(tokenized.items(), '\n')
print(tokenized['train'][8]['ids'])



dict_items([('train', Dataset({
    features: ['text'],
    num_rows: 36000
})), ('val', Dataset({
    features: ['text'],
    num_rows: 4000
}))]) 

dict_items([('train', Dataset({
    features: ['ids', 'len'],
    num_rows: 36000
})), ('val', Dataset({
    features: ['ids', 'len'],
    num_rows: 4000
}))]) 

[1722, 611, 262, 10861, 6838, 329, 543, 314, 30701, 50256]


In [3]:
print(tokenized.items())

dict_items([('train', Dataset({
    features: ['ids', 'len'],
    num_rows: 36000
})), ('val', Dataset({
    features: ['ids', 'len'],
    num_rows: 4000
}))])
